## A Brief Introduction of hidden Markov
parameters: 

1. all possible states: $ Q = \{q_1,q_2,\cdots,q_N\} $ 

2. all visible index: $ V = \{v_1, v_2, \cdots, v_M\} $ 

N stands for numbers of possible states while M stands for the numbers of visible index.

3. invisible states sequence: $ I = (i_1,i_2,\cdots,i_T) $

4. visible states sequence: $ O = (o_1,o_2,\cdots,o_T) $

5. state transfer matrix: $ A = [a_{ij}]_{N \times N} $, $ a_{ij}=P(i_{t+1}=q_j\mid i_t=q_i), i = 1,2,\cdots,N;j=1,2,\cdots,N$, which indicates the possibility of process from state $q_i$ to state $q_j$. 

6. visible index possibility matrix: $ B=[b_j(k)]_{N \times N} $,$b_j(k)=P(o_t=v_k \mid i_t=q_j), k=1,2,\cdots,M;j=1,2,\cdots,N $.

7. the initial possibility vector: $ \pi = (\pi_i) $, $\pi_i=P(i_i=q_i), i=1,2,\cdots,N $


## 前向算法

In [1]:
# test data
import numpy as np
O = np.array([0,1,0])
Q = np.array([1,2,3])
V = np.array([0,1])
PI = np.array([0.2,0.4,0.4]) 
A = np.array([[0.5,0.2,0.3],  #第一列是Q=1转变到Q=1的概率。其他元素以此类推。
              [0.3,0.5,0.2],
              [0.2,0.3,0.5]])
B = np.array([[0.5, 0.5],   # 第一列是V=0的概率，第二列是V=1的概率。
            [0.4, 0.6],
            [0.7, 0.3]])

In [2]:
container_front = [0 for i in range(len(O))]
container_front[0] = np.array([B[i, O[0]] for i in range(len(Q))])*PI
for i in range(1, len(O)):
    container_front[i] = (container_front[i-1].dot(A))*np.array([B[j,O[i]] for j in range(len(Q))])
container_front = np.array(container_front)
np.sum(container_front[-1])

0.130218

In [3]:
container_front

array([[0.1     , 0.16    , 0.28    ],
       [0.077   , 0.1104  , 0.0606  ],
       [0.04187 , 0.035512, 0.052836]])

## 后向算法

In [4]:
container_back = [0 for i in range(len(O))]
container_back[-1] = np.ones(len(Q))
for i in range(len(O)-2, -1, -1):
    container_back[i] = (container_back[i+1]*np.array([B[j,O[i]] for j in range(len(Q))])).dot(A.T)
container_back = np.array(container_back)
np.sum(np.array([B[i, O[0]] for i in range(len(Q))])*PI*container_back[0])

0.133758

In [5]:
container_back

array([[0.2461, 0.2312, 0.2577],
       [0.46  , 0.51  , 0.43  ],
       [1.    , 1.    , 1.    ]])

## 一些概率和期望值的计算

In [16]:
_ = np.array([1,2,3])
container_back/_[:,np.newaxis]

array([[0.2461    , 0.2312    , 0.2577    ],
       [0.23      , 0.255     , 0.215     ],
       [0.33333333, 0.33333333, 0.33333333]])

In [15]:
_[:,np.newaxis]

array([[1],
       [2],
       [3]])

`container_proba` 表示给定模型和观测序列O，在时刻1~t处于状态$q_i$的概率，计算公式如下：$$\gamma_t(i)=P(i_t=q_i \mid O,\lambda) =\displaystyle\frac{P(i_t=q_i,O\mid\lambda)}{P(O\mid\lambda})  = \displaystyle\frac{\alpha_t(i)\beta_t(i)}{\sum\limits_{j=1}^N{\alpha_t(j)\beta_t(j)}}$$
本程序中，最后集成为一个行为T，列为时间N的矩阵。

In [19]:
_ = np.sum(container_back*container_front, axis=1)
container_proba = container_back*container_front/_[:,np.newaxis]
container_proba = np.array(container_proba)

In [20]:
container_proba

array([[0.183989  , 0.27655916, 0.53945185],
       [0.30072507, 0.47803569, 0.22123924],
       [0.32153773, 0.27271191, 0.40575036]])

`container_conditional_proba`给定模型和观测序列O，在时刻t处于$q_i$且在t+1时刻处于$q_{j}$的概率，计算公式如下：$$ \xi_t(i,j) = \displaystyle\frac{\alpha_t(i)a_{ij}b_j(o_{t+1})\beta_{t+1}(j))}{\sum\limits_{i=1}^N\sum\limits_{j=1}^N{\alpha_t(i)a_{ij}b_j(o_{t+1})\beta_{t+1}(j))}} $$

In [50]:
container_conditional_proba = [0 for i in range(len(O)-1)]
for i in range((len(O)-1)):
    container_conditional_proba[i] = (container_front[i,np.newaxis,:].dot(A)).T.dot((np.array([B[j,O[i+1]] for j in range(len(Q))])*container_back[i+1])[np.newaxis,:])
    temp = np.sum(container_conditional_proba[i])
    container_conditional_proba[i]/=temp
container_conditional_proba = np.array(container_conditional_proba)

In [51]:
container_conditional_proba

array([[[0.09863548, 0.13122807, 0.05532164],
        [0.11785018, 0.15679198, 0.06609858],
        [0.129379  , 0.17213033, 0.07256475]],

       [[0.10551915, 0.08441532, 0.14772681],
        [0.11186996, 0.08949597, 0.15661794],
        [0.09511089, 0.07608871, 0.13315524]]])

在观测O下状态i出现的期望值

In [48]:
np.sum(container_proba, axis=0)/container_proba.shape[0]

array([0.2687506 , 0.34243559, 0.38881381])

计算观测O下，由状态i转移到状态j的期望值。

In [46]:
np.sum(container_conditional_proba, axis=0)/container_conditional_proba.shape[0]

array([[0.10207732, 0.1078217 , 0.10152423],
       [0.11486007, 0.12314397, 0.11135826],
       [0.11224495, 0.12410952, 0.10285999]])

## 学习算法
### 监督学习算法

#### 转移概率$a_{ij}$的估计
记$A_{ij}$为t时刻状态i转化为t+1时刻状态j的频数。可以得到估计参数：
$$\hat{a}_{ij}=\displaystyle\frac{A_{ij}}{\sum\limits_{j=1}^N{A_{ij}}}$$
#### 观测概率$b_{ij}$的估计
记$B_{ij}$为t时刻状态i转化为t+1时刻状态j的频数。可以得到估计参数：
$$\hat{b}_{ij}=\displaystyle\frac{B_{ij}}{\sum\limits_{j=1}^N{B_{ij}}}$$
## Baum-Welch 算法

In [55]:
####

## 预测模型
### 近似算法（贪心）

In [56]:
###

### 维特比算法（动态规划）